<a href="https://colab.research.google.com/github/Victorlouisdg/simulators/blob/main/conjugate_gradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
np.random.seed(0)

In [2]:
def iterate_cg(A, x, r, p):
    """
    Basic iteration of the conjugate gradient algorithm

    Parameters:
    x: current iterate
    r: current residual
    p: current direction
    A: matrix of interest
    """

    Ap = A @ p
    rTr = np.dot(r, r)

    a = rTr / np.dot(p, Ap)
    x_new = x + a * p
    r_new = r + a * Ap

    # New p not needed for last iteration
    beta_new = np.dot(r_new, r_new) / rTr
    p_new = -r_new + beta_new * p

    return x_new, r_new, p_new


def run_conjugate_gradient(A, x0, b, max_iter=40):
    """
    Conjugate gradient algorithm

    Parameters:
    x0: initial point
    A: matrix of interest
    b: vector in linear system (Ax = b)
    max_iter: max number of iterations to run CG
    """

    # initial iteration
    xk = x0
    rk = A @ xk - b
    pk = -rk

    directions = [pk]

    for i in (range(max_iter)):
        xk, rk, pk = iterate_cg(A, xk, rk, pk)
        e = np.sum(np.abs(rk))
        print(e, "   ", np.sum(A @ xk - b))

        directions.append(pk)

    return directions

    pk = -rk

In [3]:
n = 50

A = np.identity(n)
A *= np.random.rand(n)

x0 = np.ones(n)
b = np.random.rand(n)

directions = run_conjugate_gradient(A, x0, b)


# error = how far we are from the solution x
# residual = how far Ax is from b

7.715430637257627     -6.3349154853438545
11.162950873920975     -2.3418185592504557
9.358631407528712     1.63532454741279
7.029308633952032     0.9208721855641002
6.527974551648756     -0.0663242243141961
4.9760794716582115     0.15293991541694046
3.5280528860719507     -0.4058441770762788
2.319860607947089     -0.04547620038907216
2.0145632724361406     0.1479219538319713
1.182030340562115     0.11073720661931376
0.800327012805395     0.014843795825131595
0.39019226922741596     -0.027795685223465986
0.17899860747612845     -0.04229233424426922
0.15394265231436655     -0.006079697980735508
0.1309517469773319     0.024503309415939502
0.09281307797754822     0.0078531697427961
0.08068186002336536     -0.00887574966125194
0.10774140504415071     -0.016008698048967866
0.053629076283923896     0.0036865959597398665
0.038783108225437006     0.01050849843010967
0.02576450887996202     -0.0003568932176055183
0.009186039591557634     -0.00017422315001524354
0.004476445848841198     -0.000172

In [4]:
# Conjugancy check
for i in range(len(directions)):
    for j in range(len(directions)):
        if i != j:
            di = directions[i]
            dj = directions[j]
            print(np.dot(di, A @ dj))

-8.067851794756799e-16
9.295557567756435e-16
-1.001882197840925e-15
7.844198653344818e-16
-4.810897958663371e-16
3.725152486594134e-16
-2.9819388095009625e-16
2.608919957511935e-16
-3.272529267367855e-16
1.6311416197774612e-16
-6.706052352417903e-18
-4.7108585092116476e-17
3.650687419324926e-17
-2.694307975227841e-17
3.54185708749225e-17
-2.4612722746259948e-17
1.6015707568187684e-17
-4.6554228145299477e-17
4.702892394459098e-17
-3.5230950938676225e-17
3.055878541899732e-17
-2.485562243133994e-17
9.38885011999423e-18
-1.997919907347717e-18
-2.7653691613257394e-19
3.2864913887676123e-18
-3.230277382755506e-18
1.873825076386747e-18
-1.2486711544103074e-18
4.906314498247289e-19
-1.9078719256529212e-19
9.460627673466594e-20
-5.07137283522726e-20
2.7662908359987594e-20
-1.3925853595598757e-20
1.2046405841013946e-20
-1.0190279962522152e-20
3.1471601609240535e-21
-5.354411047547065e-21
-1.5718679637917818e-22
-8.605592361455103e-16
-3.7735484806727483e-16
4.7474654648514455e-16
-2.33914405701

In [5]:
def iterate_pcg(A, x, r, p):
    """
    Basic iteration of the conjugate gradient algorithm

    Parameters:
    x: current iterate
    r: current residual
    p: current direction
    A: matrix of interest
    """

    Ap = A @ p
    rTr = np.dot(r, r)

    a = rTr / np.dot(p, Ap)
    x_new = x + a * p
    r_new = r + a * Ap

    # New p not needed for last iteration
    beta_new = np.dot(r_new, r_new) / rTr
    p_new = -r_new + beta_new * p

    return x_new, r_new, p_new


def run_preconditioned_conjugate_gradient(A, x, b, max_iter=40):
    """
    Conjugate gradient algorithm

    Parameters:
    x0: initial point
    A: matrix of interest
    b: vector in linear system (Ax = b)
    max_iter: max number of iterations to run CG
    """

    # initial iteration
    r = b - A @ x

    M_inv = np.identity(x.shape[0]) / np.diagonal(A)

    # M_inv = np.identity(x.shape[0])

    d = M_inv @ r

    delta_new = np.dot(r, d)
    delta_0 = delta_new

    for i in (range(max_iter)):
        Ad = A @ d
        alpha = delta_new / np.dot(d, Ad)
        x = x + alpha * d
        r = r - alpha * Ad
        s = M_inv @ r
        delta_old = delta_new
        delta_new = np.dot(r, s)
        beta = delta_new / delta_old
        d = s + beta * d
        
        e = np.sum(np.abs(r))
        print(e, "   ", np.sum(A @ x - b))

        if e < 10e-5:
            break



n = 50

A = np.identity(n)
A *= np.random.rand(n)

# A = np.random.rand(n*n).reshape(n,n) + 5 * np.identity(n)

x0 = np.ones(n)
b = np.random.rand(n)

run_preconditioned_conjugate_gradient(A, x0, b)

6.869504964868156e-16     -1.0408340855860843e-17
